In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas datetime matplotlib seaborn neuralprophet scikit-learn torch wxPython statsmodels openpyxl kaleido --upgrade plotly

  Using cached pandas-2.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached matplotlib-3.10.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached neuralprophet-0.8.0-py3-none-any.whl.metadata (9.1 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached torch-2.8.0-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 4.8 MB/s  0:00:02eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 23.5 MB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 22.9 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 21.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
pip freeze

absl-py==2.3.1
aiohappyeyeballs==2.6.1
aiohttp==3.12.15
aiosignal==1.4.0
appnope==0.1.4
asttokens==3.0.0
attrs==25.3.0
captum==0.8.0
comm==0.2.3
contourpy==1.3.3
cycler==0.12.1
DateTime==5.5
debugpy==1.8.16
decorator==5.2.1
executing==2.2.1
fastjsonschema==2.21.2
filelock==3.19.1
fonttools==4.59.2
frozenlist==1.7.0
fsspec==2025.9.0
grpcio==1.74.0
holidays==0.80
idna==3.10
ipykernel==6.30.1
ipython==9.5.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
Jinja2==3.1.6
joblib==1.5.2
jsonschema==4.25.1
jsonschema-specifications==2025.9.1
jupyter_client==8.6.3
jupyter_core==5.8.1
kiwisolver==1.4.9
lightning-utilities==0.15.2
Markdown==3.9
MarkupSafe==3.0.2
matplotlib==3.10.6
matplotlib-inline==0.1.7
mpmath==1.3.0
multidict==6.6.4
nbformat==5.10.4
nest-asyncio==1.6.0
networkx==3.5
neuralprophet==0.8.0
numpy==1.26.4
packaging==25.0
pandas==2.3.2
parso==0.8.5
pexpect==4.9.0
pillow==11.3.0
platformdirs==4.4.0
plotly==5.24.1
prompt_toolkit==3.0.52
propcache==0.3.2
protobuf==6.32.1
psutil==7.0.0
ptypr

In [40]:
import io
import os
import numpy as np
import pandas as pd
import warnings
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.feature_selection import SelectFromModel
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.api import VAR
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tools.eval_measures import aic, bic
from datetime import datetime
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from hyperopt import hp, Trials
from hyperopt import fmin, tpe
from hyperopt import STATUS_OK
from dateutil.relativedelta import relativedelta
import ipywidgets as widgets
import torch  
import statsmodels.api as sm
from pandas import pivot_table
from pandas._libs.tslibs.parsing import DateParseError
import matplotlib 
import matplotlib.pyplot as plt
from IPython.display import display
from tabulate import tabulate
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import gui
from dateutil.relativedelta import relativedelta
import GrangerCausality
import NP_model

## Lectura de bases

In [ ]:
df = gui.select_file_via_gui(window_title="Seleccione el archivo con datos historicos")
if df is not None:
    col_fecha = gui.select_column_from_dataframe(df, window_title="Seleccione la columna de fecha")
    if col_fecha:
        print(f"Selected column: {col_fecha}")
    else:
        print("No column selected.")

    col_pred = gui.select_column_from_dataframe(df, window_title="Seleccione la columna para la predicción")
    if col_pred:
        print(f"Selected column: {col_pred}")
    else:
        print("No column selected.")
else:
    print("No file selected or failed to load.")


VarsEco_Base = gui.select_file_via_gui(window_title="Seleccione el archivo con las variables economicas (Escenario Base)")
if VarsEco_Base is not None:
    VarsEco_Base.head()
else:
    print("No file selected or failed to load.")

VarsEco_Adv = gui.select_file_via_gui(window_title="Seleccione el archivo con las variables economicas (Escenario Adverso)")
if VarsEco_Adv is not None:
    VarsEco_Adv.head()
else:
    print("No file selected or failed to load.")

In [ ]:
#Definimos columnas de añomes como strings
try:
    df['aniomes'] = pd.to_datetime(df[col_fecha]).dt.strftime('%Y%m')
except DateParseError as e:
    if len(df[col_fecha].max()) != 6:
        print("Error: Revisar el formato de la columna de fecha. Asegurar que venga en formato de añomes(AAAAMM) o de fecha-hora.")
        
VarsEco_Base['aniomes'] = VarsEco_Base['aniomes'].astype(str)
VarsEco_Adv['aniomes'] = VarsEco_Adv['aniomes'].astype(str)
fecha_real = df['aniomes'].max()

Eco_Base = VarsEco_Base[VarsEco_Base['aniomes']<= fecha_real]
Eco_Base.head()

Eco_Adv = VarsEco_Adv[VarsEco_Adv['aniomes']<= fecha_real]
Eco_Adv.head()

same = (Eco_Base.columns==Eco_Adv.columns).all()
if same:
    print("The column names have the same order.")
else:
    print("The column names do not have the same order.")


Bs_Hist=df.merge(Eco_Base, how='left', on='aniomes')
Bs_Hist.head()

In [42]:
forecast=len(VarsEco_Base)-len(Eco_Base)
forecast

30

In [55]:
# Additional variables to include in all DataFrames
additional_vars = ['Fondeo1dia', 'Cetes28', 'Cetes91', 'Cetes182', 'Cetes364', 
                   'BonoM3', 'BonoM5', 'BonoM10', 'TasaFedEUA', 'Tbill1m', 'Tbill3m', 
                   'Tbill6m', 'Tbill12m', 'Tnote3A', 'Tnote5A', 'Tnote10A', 'InflacionAn', 
                   'MXNaUSD', 'USDaEUR', 'CambioPIB_anual', 'Desempleo', 'IPC', 
                   'SyP', 'ExpNoPetro', 'VIX_BMV', 'VIX_USA']

In [43]:
fecha_train=input("Ingrese el añomes para hacer el corte de train/test")

In [56]:
df_inicial = Bs_Hist[['aniomes', col_pred] + additional_vars].reset_index(drop=True).copy()
df_inicial.head()

In [59]:
# Rename the selected columns
df_inicial.rename(columns={'aniomes':'ds'},inplace=True)
df_inicial.rename(columns={col_pred:'y'},inplace=True)
df_inicial['ds']=(df_inicial['ds'].astype(str).str[0:4])+'-'+(df_inicial['ds'].astype(str).str[4:6])+'-'+'01'
df_inicial['ds']=pd.to_datetime(df_inicial['ds'])
df_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ds               895 non-null    datetime64[ns]
 1   y                895 non-null    int64         
 2   Fondeo1dia       895 non-null    float64       
 3   Cetes28          895 non-null    float64       
 4   Cetes91          895 non-null    float64       
 5   Cetes182         895 non-null    float64       
 6   Cetes364         895 non-null    float64       
 7   BonoM3           895 non-null    float64       
 8   BonoM5           895 non-null    float64       
 9   BonoM10          895 non-null    float64       
 10  TasaFedEUA       895 non-null    float64       
 11  Tbill1m          895 non-null    float64       
 12  Tbill3m          895 non-null    float64       
 13  Tbill6m          895 non-null    float64       
 14  Tbill12m         895 non-null    float64  

: 

In [ ]:
date_train = datetime.strptime(str(fecha_train),"%Y%m")
date_train = datetime.strftime(date_train,"%Y-%m-01")
date_train = pd.to_datetime(date_train)

df_train = df_inicial[df_inicial['ds']<= date_train]
df_train.tail()

## Pruebas de Granger

In [ ]:
stationary_info = GrangerCausality.make_series_stationary(df_train)

# Create a new DataFrame with differenced variables
differenced_df = GrangerCausality.make_differenced_dataframe(df_train, stationary_info)


# Perform Granger causality test with automatic lag selection based on SSR Chi-Square test
granger_results = GrangerCausality.granger_causality_test(differenced_df, 'y', test='ssr_chi2test')

print("Granger Causality Test Results using SSR Chi-Square Test:")
for variable, result in granger_results.items():
    print(f"{variable}: p-value = {result['p_value']}, Best Lag = {result['best_lag']}")

In [ ]:
# Extract significant variables and their p-values
significant_variables = [(variable, result['p_value']) for variable, result in granger_results.items() if result['p_value'] <= 0.05]

# Create a DataFrame from the list
tabla1=pd.DataFrame(significant_variables, columns=['Variable', 'p-value'])

tabla1
# Print the results
#print("Significant Variables (p-value <= 0.05):")
#for variable, p_value in significant_variables_list:
#    print(f"{variable}: p-value = {p_value}")

# Extract significant variables as a list
significant_variables_list = [variable for variable, result in granger_results.items() if result['p_value'] <= 0.05]
future_regressor=significant_variables_list
future_regressor

In [1]:
import pandas as pd
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank'],
        'Age': [25, 30, 35, 40, 45, 50],
        'City': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney', 'Rome']}
df = pd.DataFrame(data)

In [ ]:
df_red=Bs_Hist[['ds','y'] + future_regressor]
df_red.head()

## Entrenamiento

In [44]:
train_end_date = date_train
test_start_date = train_end_date + pd.DateOffset(months=1)

fecha_real_formato = datetime.strptime(str(fecha_real),"%Y%m")
fecha_real_formato = datetime.strftime(fecha_real_formato,"%Y-%m-01")
fecha_real_formato = pd.to_datetime(fecha_real_formato)

rel = relativedelta(train_end_date, fecha_real_formato)
forecast = abs(rel.years * 12 + rel.months)

In [ ]:
basic=df_red[['ds','y']+ future_regressor]

# Filter the data based on the cutoff date
df_train = basic[basic['ds'] < test_start_date]
df_test = basic[basic['ds'] >= test_start_date]

In [ ]:
next_months_df= basic[basic['ds'] >= test_start_date]
next_months_df=next_months_df[['ds']+ future_regressor]
next_months_df['y']=None
next_months_df=next_months_df[['ds','y']+ future_regressor]
df_train_f=pd.concat([df_train,next_months_df],ignore_index=True)
df_train_f

In [ ]:
lr = NP_model.find_best_lr(basic, test_start_date, forecast, future_regressor)

In [ ]:
# Split the dataset into train and test sets
df_train = basic[basic['ds'] < test_start_date].copy()
df_test = basic[basic['ds'] >= test_start_date].copy()

In [ ]:
#Evaluates and saves all errors to a single CSV file for later analysis
NP_model.regressor_error(basic, df_train, df_test, lr, test_start_date, forecast, future_regressor)

#Model training
importance_scores = NP_model.get_importance_scores(basic, df_train, df_test, lr, test_start_date, forecast, future_regressor)

In [ ]:
# Prepare data for plotting
variables = [var for var, score in importance_scores]
scores = [score for var, score in importance_scores]

# Create a DataFrame for seaborn
df_importance = pd.DataFrame({'Variable': variables, 'Importance Score': scores})

# Create a horizontal bar plot
fig = plt.figure(figsize=(10, 6))
sns.barplot(x='Importance Score', y='Variable', data=df_importance, palette='viridis')
plt.xlabel('Importance Score')
plt.ylabel('Variable')
plt.title('Variable Importance Scores')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()
fig.savefig('Importance_Scores.png', dpi=fig.dpi)

In [ ]:
m, metrics = NP_model.train_model(df_train, df_test, lr, forecast, future_regressor)

In [ ]:
# Create a larger figure
fig = plt.figure(figsize=(12, 6))

# Plot training loss curve
plt.plot(metrics['Loss'], label='Training Loss', color='blue')

# Plot validation loss curve
plt.plot(metrics['Loss_val'], label='Validation Loss', color='orange')

# Add labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Show the plot
plt.show()

fig.savefig('Model_Loss.png', dpi=fig.dpi)

In [ ]:
m.plot_parameters().savefig('Model_Parameters.png')

In [ ]:
for variable, result in granger_results.items():
    if variable in future_regressor:
        print(f"{variable}: Best Lag = {result['best_lag']}")

In [ ]:
df_train_forecast=m.predict(df_train_f)
# Visualize the forecast
m.plot(df_train_forecast)

In [ ]:
dd= df_train_forecast.merge(basic[['ds','y']], left_on='ds', right_on='ds', how='outer')
dd[['ds','y_y','yhat1']].tail(forecast+1)

In [ ]:
# Extract 'ds', 'yhat1', and 'y_y' columns from the DataFrame
ds = dd['ds']
yhat1 = dd['yhat1']
y_y = dd['y_y']

# Plot the data
fig = plt.figure(figsize=(10, 6))
plt.plot(ds, yhat1, label='yhat1', color='blue')
plt.plot(ds, y_y, label='y_y', color='green')
plt.axvline(x=train_end_date, color='red', linestyle='--', label='train_end_date')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Comparison of yhat1 and y_y')
plt.legend()
plt.show()
fig.savefig('Comparacion.png', dpi=fig.dpi)

## Modelo

In [ ]:
print(VarsEco_Base.aniomes.max())
print(VarsEco_Adv.aniomes.max())

In [ ]:
Esc_Base=VarsEco_Base.loc[VarsEco_Base['aniomes']>fecha_real,['aniomes']+future_regressor]
# Rename the selected columns
Esc_Base.rename(columns={'aniomes':'ds'},inplace=True)
Esc_Base['ds']=(Esc_Base['ds'].astype(str).str[0:4])+'-'+(Esc_Base['ds'].astype(str).str[4:6])+'-'+'01'
Esc_Base['ds']=pd.to_datetime(Esc_Base['ds'])
Esc_Base['y']=None
Esc_Base=Esc_Base[['ds','y']+ future_regressor]
df_base=pd.concat([basic,Esc_Base],ignore_index=True)
#df_base['covid']=df_base['covid'].fillna(0)
df_base.tail()

In [ ]:
forecast = len(Esc_Base)
forecast

In [ ]:
metrics = m.fit(basic,freq='MS');
metrics

In [ ]:
df_base_forecast=m.predict(df_base)

In [ ]:
m.plot(df_base_forecast)

In [ ]:
print(basic.loc[basic['y']==basic['y'].max(),'ds'])
print(basic.loc[basic['y']==basic['y'].min(),'ds'])

In [ ]:
Esc_Adv=VarsEco_Adv.loc[VarsEco_Adv['aniomes']>fecha_real,['aniomes']+future_regressor]
# Rename the selected columns
Esc_Adv.rename(columns={'aniomes':'ds'},inplace=True)
Esc_Adv['ds']=(Esc_Adv['ds'].astype(str).str[0:4])+'-'+(Esc_Adv['ds'].astype(str).str[4:6])+'-'+'01'
Esc_Adv['ds']=pd.to_datetime(Esc_Adv['ds'])
Esc_Adv['y']=None
Esc_Adv=Esc_Adv[['ds','y']+ future_regressor]
df_Adv=pd.concat([basic,Esc_Adv],ignore_index=True)
#df_Adv['covid']=df_Adv['covid'].fillna(0)
df_Adv.tail()

In [ ]:
df_Adv_forecast=m.predict(df_Adv)

In [ ]:
m.plot(df_Adv_forecast)

In [ ]:
df_base_forecast.rename(columns={'yhat1':'y_Base'},inplace=True)
df_Adv_forecast.rename(columns={'yhat1':'y_Adv'},inplace=True)

In [ ]:
MiPyMes1Estres= df_base_forecast.merge(df_Adv_forecast[['ds','y_Adv']], left_on='ds', right_on='ds', how='outer')

In [ ]:
# Extract 'ds', 'yhat1', and 'y_y' columns from the DataFrame
ds = MiPyMes1Estres['ds']
yBase = MiPyMes1Estres['y_Base']
yAdv = MiPyMes1Estres['y_Adv']
yReal = MiPyMes1Estres['y']

# Plot the data
fig = plt.figure(figsize=(10, 6))
plt.plot(ds, yBase, label='yBase', color='blue')
plt.plot(ds, yAdv, label='yAdv', color='green')
plt.plot(ds, yReal, label='yReal', color='black')
plt.axvline(x=basic.ds.max(), color='red', linestyle='--', label='real')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Comparison of MiPyMes Etapa 1')
plt.legend()
plt.show()
fig.savefig('Forecast.png', dpi=fig.dpi)

In [1]:
from lightning.pytorch import Trainer

# To save logs and checkpoints to a specific local directory
trainer = Trainer(default_root_dir="/your/path/to/save/experiments")

ModuleNotFoundError: No module named 'lightning'